In [30]:
import pandas as pd
import numpy as np

## Student Counts

In [31]:
df = pd.read_csv("~/SFUSD/Data/Cleaned/drop_optout_1819.csv", low_memory=False)
df = df.loc[df.grade == "KG"]
assert len(df) == 4772

In [32]:
siblings = pd.read_csv("klm_raw_inputs/preassigned_siblings.csv")
print(len(siblings))
df = pd.merge(df, siblings, on="studentno", how="outer", indicator=True)
df = df[df['_merge'] == 'left_only']
assert len(df) == 4772 - 1290 
# df

1290


In [33]:
df = df.dropna(subset=["census_blockgroup"])
df.census_blockgroup = df.census_blockgroup.astype(int)
df.loc[:, "lang_match"] = df.homelang_desc.apply(
    lambda x: "cantonese"
    if x == "CC-Chinese Cantonese"
    else ("spanish" if x == "SP-Spanish" else "no_match")
)

counts = (
    df.groupby(["census_blockgroup", "lang_match"], as_index=False)
    .count()
    .sort_values(["census_blockgroup", "lang_match"])
    .rename(columns={"studentno": "student_count"})[
        ["census_blockgroup", "lang_match", "student_count"]
    ]
)
counts.index.name = "type_code"
counts

,census_blockgroup,lang_match,student_count
type_code,,,
0,60750101001,no_match,2
1,60750101002,cantonese,2
2,60750101002,no_match,5
3,60750101002,spanish,2
4,60750102001,no_match,3
...,...,...,...
917,60759805011,cantonese,1
918,60759805011,no_match,3
919,60759806001,no_match,4


In [34]:
# for rnd in range(2, 6):
#     if f"r{rnd}_programs" in df.columns:
#         df["r1_programs"] = df["r1_programs"].fillna(df[f"r{rnd}_programs"])
# df.r1_programs = df.r1_programs.apply(lambda x: eval(x) if not pd.isna(x) else [])
# df["ge_students"] = df.apply(
#     lambda x: sum([i == "GE" for i in x["r1_programs"]]) / len(x["r1_programs"])
#     if  len(x["r1_programs"]) > 0
#     else 0,
#     axis=1
# )

In [35]:
# counts = (
#     df[["census_blockgroup"]]
#     .groupby("census_blockgroup", as_index=False)
#     .sum()
#     .sort_values("census_blockgroup")
# )

# counts.index.name = "geocode"

In [36]:
# assert len(counts) == 931
assert len(counts) == 922
counts.to_csv("klm_raw_inputs/geocode_student_counts.csv")

In [37]:
ref = counts[["census_blockgroup", "lang_match"]]
ref.to_csv("klm_raw_inputs/typecodes.csv")
len(counts)

922

## Block Group FRL

In [38]:
df["frl"] = df.reducedlunch_prob + df.freelunch_prob
frl = (
    df[["frl", "census_blockgroup"]]
    .groupby("census_blockgroup", as_index=False)
    .first()
    .sort_values("census_blockgroup")
)
frl = ref.merge(frl, how="left", on="census_blockgroup")

frl.index.name = "type_code"

In [39]:
frl

,census_blockgroup,lang_match,frl
type_code,,,
0,60750101001,no_match,0.100000
1,60750101002,cantonese,0.454545
2,60750101002,no_match,0.454545
3,60750101002,spanish,0.454545
4,60750102001,no_match,0.000000
...,...,...,...
917,60759805011,cantonese,0.780822
918,60759805011,no_match,0.780822
919,60759806001,no_match,0.750000


In [40]:
assert np.equal(ref.census_blockgroup.to_numpy(), frl.census_blockgroup.to_numpy()).all()
frl[["frl"]].to_csv("klm_raw_inputs/geocode_frl.csv")

## School Capacities

~~REDO WITH REMAINING CAPACITY AFTER SIBLING ASSIGNMENT~~ ==> now done in sfusd_preassign_siblings

In [41]:
# progs = pd.read_csv("~/SFUSD/Data/Cleaned/programs_1819.csv")

In [42]:
# progs = progs.loc[progs.program_type == "GE"]
# caps = progs.sort_values("school_id").reset_index()[["school_id", "capacity"]]
# caps

In [43]:
# caps.to_csv("klm_raw_inputs/school_caps.csv")

# Block to School Distances

In [44]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()

In [45]:
path = "~/SFUSD/Census 2010_ Blocks for San Francisco/geo_export_d4e9e90c-ff77-4dc9-a766-6a1a7f7d9f9c.shp"
sf = gpd.read_file(os.path.expanduser(path))
sf['geoid10'] = sf['geoid10'].fillna(value=0).astype('int64', copy=False)
translator = pd.read_csv('~/Dropbox/SFUSD/Optimization/block_blockgroup_tract.csv')
translator['Block'] = translator['Block'].fillna(value=0).astype('int64', copy=False)
sf = sf.merge(translator,how='left',left_on='geoid10',right_on='Block')
sf = sf.dissolve(by="BlockGroup", as_index=False).to_crs(epsg=3395)

In [46]:
sf["center"] = sf.geometry.centroid
block_loc = sf.sort_values("BlockGroup").rename(columns={"BlockGroup": "census_blockgroup"})[["census_blockgroup", "center"]]
block_loc = gpd.GeoDataFrame(block_loc, geometry="center").to_crs(epsg=4326)
block_loc = ref.merge(block_loc, how="left", on="census_blockgroup")
assert np.equal(counts.census_blockgroup.to_numpy(), block_loc.census_blockgroup.to_numpy()).all()
block_loc

,census_blockgroup,lang_match,center
0,60750101001,no_match,POINT (-122.41178 37.80773)
1,60750101002,cantonese,POINT (-122.41016 37.80510)
2,60750101002,no_match,POINT (-122.41016 37.80510)
3,60750101002,spanish,POINT (-122.41016 37.80510)
4,60750102001,no_match,POINT (-122.42055 37.80497)
...,...,...,...
917,60759805011,cantonese,POINT (-122.41869 37.71794)
918,60759805011,no_match,POINT (-122.41869 37.71794)
919,60759806001,no_match,POINT (-122.36847 37.72535)
920,60759806001,spanish,POINT (-122.36847 37.72535)


In [47]:
progs = pd.read_csv("~/SFUSD/Data/Cleaned/programs_1819.csv")
progs = progs.sort_values("program_id").reset_index(drop=True)[["program_id", "school_id", "program_type"]]
progs.index.name = "program_idx"
progs[["program_id"]].to_csv("klm_raw_inputs/program_idxs.csv")
progs

,program_id,school_id,program_type
program_idx,,,
0,413-GE-KG,413,GE
1,413-SA-KG,413,SA
2,420-GE-KG,420,GE
3,420-MS-KG,420,MS
4,420-SE-KG,420,SE
...,...,...,...
154,872-CB-KG,872,CB
155,872-GE-KG,872,GE
156,876-CE-KG,876,CE


In [48]:
sch = pd.read_csv("~/SFUSD/Data/Cleaned/schools_rehauled_1819.csv")
sch = progs[["school_id", "program_id"]].merge(sch, how="left", on="school_id")
assert np.equal(sch.program_id.to_numpy(), progs.program_id.to_numpy()).all() 
sch

,school_id,program_id,school_name,school_name_long,lat,lon,zip,category,grades,cap_lb,...,ela_color,math_color,chronic_color,suspension_color,MetStandards,AvgColorIndex,attendance_area,Block,BlockGroup,Tract
0,413,413-GE-KG,Alamo ES,Alamo Elementary School,37.783001,-122.482263,94121,Attendance,K-5th,96,...,Green,Green,Orange,Blue,0.614786,3.75,413,60750426011000,60750426011,6.075043e+09
1,413,413-SA-KG,Alamo ES,Alamo Elementary School,37.783001,-122.482263,94121,Attendance,K-5th,96,...,Green,Green,Orange,Blue,0.614786,3.75,413,60750426011000,60750426011,6.075043e+09
2,420,420-GE-KG,Alvarado ES (TK-5),Alvarado Elementary School,37.753715,-122.438162,94114,Attendance,TK-5th,93,...,Green,Green,Orange,Blue,0.519149,3.75,420,60750212003004,60750212003,6.075021e+09
3,420,420-MS-KG,Alvarado ES (TK-5),Alvarado Elementary School,37.753715,-122.438162,94114,Attendance,TK-5th,93,...,Green,Green,Orange,Blue,0.519149,3.75,420,60750212003004,60750212003,6.075021e+09
4,420,420-SE-KG,Alvarado ES (TK-5),Alvarado Elementary School,37.753715,-122.438162,94114,Attendance,TK-5th,93,...,Green,Green,Orange,Blue,0.519149,3.75,420,60750212003004,60750212003,6.075021e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,872,872-CB-KG,Chin ES,John Yehall Chin Elementary School,37.798611,-122.403217,94133,Attendance,K-5th,46,...,Blue,Blue,Green,Blue,0.638298,4.75,872,60750106002000,60750106002,6.075011e+09
155,872,872-GE-KG,Chin ES,John Yehall Chin Elementary School,37.798611,-122.403217,94133,Attendance,K-5th,46,...,Blue,Blue,Green,Blue,0.638298,4.75,872,60750106002000,60750106002,6.075011e+09
156,876,876-CE-KG,West Portal ES,West Portal Elementary School,37.742816,-122.464389,94127,Attendance,K-5th,99,...,Green,Green,Green,Blue,0.532646,4.25,876,60750308001002,60750308001,6.075031e+09
157,876,876-CN-KG,West Portal ES,West Portal Elementary School,37.742816,-122.464389,94127,Attendance,K-5th,99,...,Green,Green,Green,Blue,0.532646,4.25,876,60750308001002,60750308001,6.075031e+09


In [49]:
sch_loc = sch[["program_id", "lat", "lon"]]

In [50]:
block_loc.loc[:, "key"] = 0
sch_loc.loc[:, "key"] = 0
dist = block_loc.merge(sch_loc, on="key")
dist.loc[:, "dist"] = dist.apply(
    lambda x: 6371.01
    * np.arccos(
        np.sin(x["lat"] * np.pi / 180) * np.sin(x["center"].y * np.pi / 180)
        + np.cos(x["lat"] * np.pi / 180)
        * np.cos(x["center"].y * np.pi / 180)
        * np.cos((x["lon"] - x["center"].x) * np.pi / 180)
    )
    * 0.621371,  # return distance in miles,
    axis=1,
)
dist = dist.pivot(index=["census_blockgroup", "lang_match"], columns="program_id", values="dist")
dist

/Users/katherinementzer/opt/anaconda3/envs/geo_env/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


program_id                    413-GE-KG  413-SA-KG  420-GE-KG  420-MS-KG  \
census_blockgroup lang_match                                               
60750101001       no_match     4.210265   4.210265   4.000365   4.000365   
60750101002       cantonese    4.222406   4.222406   3.865411   3.865411   
                  no_match     4.222406   4.222406   3.865411   3.865411   
                  spanish      4.222406   4.222406   3.865411   3.865411   
60750102001       no_match     3.695885   3.695885   3.670033   3.670033   
...                                 ...        ...        ...        ...   
60759805011       cantonese    5.680639   5.680639   2.691146   2.691146   
                  no_match     5.680639   5.680639   2.691146   2.691146   
60759806001       no_match     7.382861   7.382861   4.282489   4.282489   
                  spanish      7.382861   7.382861   4.282489   4.282489   
60759809001       no_match     5.660137   5.660137   2.682321   2.682321   

program_id                    420-SE-KG  420-SN-KG  435-GE-KG  449-FB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match     4.000365   4.000365   4.174812   2.205296   
60750101002       cantonese    3.865411   3.865411   4.157071   2.014224   
                  no_match     3.865411   3.865411   4.157071   2.014224   
                  spanish      3.865411   3.865411   4.157071   2.014224   
60750102001       no_match     3.670033   3.670033   3.670199   2.138995   
...                                 ...        ...        ...        ...   
60759805011       cantonese    2.691146   2.691146   5.063947   4.073348   
                  no_match     2.691146   2.691146   5.063947   4.073348   
60759806001       no_match     4.282489   4.282489   6.829878   4.073554   
                  spanish      4.282489   4.282489   6.829878   4.073554   
60759809001       no_match     2.682321   2.682321   5.139355   2.299547   

program_id                    449-GE-KG  453-GE-KG  ...  859-GE-KG  862-CB-KG  \
census_blockgroup lang_match                        ...                         
60750101001       no_match     2.205296   6.355311  ...   1.842501   6.844805   
60750101002       cantonese    2.014224   6.159751  ...   1.692426   6.780568   
                  no_match     2.014224   6.159751  ...   1.692426   6.780568   
                  spanish      2.014224   6.159751  ...   1.692426   6.780568   
60750102001       no_match     2.138995   6.282540  ...   1.597761   6.376376   
...                                 ...        ...  ...        ...        ...   
60759805011       cantonese    4.073348   1.618879  ...   4.416798   4.635329   
                  no_match     4.073348   1.618879  ...   4.416798   4.635329   
60759806001       no_match     4.073554   1.249711  ...   4.811351   7.229708   
                  spanish      4.073554   1.249711  ...   4.811351   7.229708   
60759809001       no_match     2.299547   1.926615  ...   2.998123   6.019972   

program_id                    862-GE-KG  867-CB-KG  867-GE-KG  872-CB-KG  \
census_blockgroup lang_match                                               
60750101001       no_match     6.844805   6.578492   6.578492   0.784621   
60750101002       cantonese    6.780568   6.396017   6.396017   0.586841   
                  no_match     6.780568   6.396017   6.396017   0.586841   
                  spanish      6.780568   6.396017   6.396017   0.586841   
60750102001       no_match     6.376376   6.412272   6.412272   1.043272   
...                                 ...        ...        ...        ...   
60759805011       cantonese    4.635329   0.592967   0.592967   5.637745   
                  no_match     4.635329   0.592967   0.592967   5.637745   
60759806001       no_match     7.229708   2.450688   2.450688   5.405716   
                  spanish      7.229708   2.450688   2.450688   5.405716   
60759809001       no_match     6.019972   2.521446   2.5

In [51]:
assert np.equal(sch.program_id.to_numpy(), np.array(dist.columns)).all()
assert np.equal(
    counts.census_blockgroup.to_numpy(),
    dist.index.get_level_values("census_blockgroup").to_numpy(),
).all()
assert np.equal(
    counts.lang_match.to_numpy(), dist.index.get_level_values("lang_match").to_numpy()
).all()

In [52]:
dist.to_csv("klm_raw_inputs/distances.csv")#, header=False, index=False)

In [53]:
counts.student_count.sum()

3452